In [1]:
import pandas as pd
import pandas_datareader as data
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
start_date = '1928-01-01'
end_date = '2022-08-01'
years = 30
tax = 0.2
init_amount = 1600000
monthly = 8000
result = {}

data_source = 'yahoo'
sap500 = '^GSPC'
dow = '^DJI'

result_df = pd.DataFrame(columns = ['start_date','months_survive','survive'])

In [3]:
# pull raw data
cpi = data.DataReader("CPIAUCNS", "fred", start_date, end_date)
stock = data.DataReader(sap500, data_source, start_date, end_date)[['Close']]


# user merger and then fill in monthly from the previous close if possible else the next close
df = cpi.merge(stock, how='outer', left_index=True, right_index=True)
df[['Close']] = df[['Close']].fillna(method='ffill').fillna(method='bfill')

# remove non first month 
df.dropna(inplace=True)

# set placeholders
df['inflation'] = 0.0
df['change'] = 0.0
df.reset_index(inplace=True)
df.head()

,index,CPIAUCNS,Close,inflation,change
0,1928-01-01,17.3,17.760000,0.0,0.0
1,1928-02-01,17.1,17.530001,0.0,0.0
2,1928-03-01,17.1,17.299999,0.0,0.0
3,1928-04-01,17.1,19.280001,0.0,0.0
4,1928-05-01,17.2,19.780001,0.0,0.0


In [4]:
for i in range(len(df)):
    # skip the first iteration
    if i == 0: continue
    # update inflation
    p_cpi = df.at[i-1,'CPIAUCNS']
    c_cpi = df.at[i,'CPIAUCNS']
    df.at[i,'inflation']= (c_cpi - p_cpi)/p_cpi
    # update percent change
    p_close = df.at[i-1,'Close']
    c_close = df.at[i,'Close']
    df.at[i,'change'] = (c_close - p_close)/p_close

In [23]:
#last start index
lsi = int((len(df)- years*12)/12)
for s in range(lsi):
    s_date_i = s*12
    s_date = df.at[s_date_i,'index'].strftime('%Y_%m_%d')
    mnth = f'monthly_{s_date}'
    prcp = f'principle_{s_date}'
    df[mnth] = np.nan
    df[prcp] = np.nan
    k=0
    end = s_date_i + years*12 +1
    start = df.at[s_date_i,'index']
    for i in range(s_date_i,end):
        if k==0:
            df.at[i,mnth]= monthly
            df.at[i,prcp]= init_amount
            k+=1
            continue
        if i > len(df):
            break
        p = i-1
        # previous monthly * (1+inflation)
        p_m = df.at[p,mnth]
        c_i = df.at[i,'inflation']
        c_m = p_m*(1+c_i)
        df.at[i,mnth]= c_m
        # (previous principle*(1+change)) - current monthly
        p_p = df.at[p,prcp]
        p_c = df.at[p,'Close']
        c_c = df.at[i,'Close']
        change =  (c_c-p_c)/p_c
        c_p = p_p*(1+change)-c_m
        df.at[i,prcp]= c_p
        if c_p <= 0 or i == end-1:
            res = pd.DataFrame.from_dict({'start_date': [start], 'months_survive': [i-s_date_i], 'survive': [c_p > 0]})
            result_df = pd.concat([result_df,res], ignore_index=True)
            break
    df.drop(columns=[mnth], inplace=True)
    

In [29]:
df.head(20)

,index,CPIAUCNS,Close,inflation,change,principle_1928_01_01,principle_1929_01_01,principle_1930_01_01,principle_1931_01_01,principle_1932_01_01,...,principle_1982_01_01,principle_1983_01_01,principle_1984_01_01,principle_1985_01_01,principle_1986_01_01,principle_1987_01_01,principle_1988_01_01,principle_1989_01_01,principle_1990_01_01,principle_1991_01_01
0,1928-01-01,17.3,17.760000,0.000000,0.000000,1.600000e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1928-02-01,17.1,17.530001,-0.011561,-0.012950,1.571372e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1928-03-01,17.1,17.299999,0.000000,-0.013120,1.542847e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1928-04-01,17.1,19.280001,0.000000,0.114451,1.711520e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1928-05-01,17.2,19.780001,0.005848,0.025934,1.747952e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1928-06-01,17.1,20.070000,-0.005814,0.014661,1.765672e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1928-07-01,17.1,19.139999,0.000000,-0.046338,1.675947e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1928-08-01,17.1,19.420000,0.000000,0.014629,1.692557e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1928-09-01,17.3,20.870001,0.011696,0.074665,1.810932e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1928-10-01,17.2,21.360001,-0.005780,0.023479,1.845497e+06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
result_df

,start_date,months_survive,survive
0,1928-01-01 00:00:00,163,False
1,1929-01-01 00:00:00,116,False
2,1930-01-01 00:00:00,127,False
3,1931-01-01 00:00:00,150,False
4,1932-01-01 00:00:00,260,False
...,...,...,...
123,1987-01-01 00:00:00,360,True
124,1988-01-01 00:00:00,360,True
125,1989-01-01 00:00:00,360,True
126,1990-01-01 00:00:00,360,True


In [30]:
sum(result_df.survive)/len(result_df)

0.390625

In [ ]:
# Need to fix

# title = f'principle = {init_amount}  withdrawals = {monthly}'
# yl='Principle [$]'

# df[['index','principle_1']].dropna().plot('index','principle_1', title=title, ylabel=yl)

128